In [1]:
from math import cos, sin, asin, sqrt, pi

import cudf
import numpy as np
from numba import cuda

In [2]:
np.random.seed(12)
data_length = 1000

df = cudf.DataFrame()
df['lat1'] = np.random.normal(10, 1, data_length)
df['lon1'] = np.random.normal(10, 1, data_length)
df['lat2'] = np.random.normal(10, 1, data_length)
df['lon2'] = np.random.normal(10, 1, data_length)

In [3]:
def haversine_distance_kernel(lat1, lon1, lat2, lon2, out):
    """Haversine distance formula taken from Michael Dunn's StackOverflow post:
    https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
    """
    for i, (x_1, y_1, x_2, y_2) in enumerate(zip(lat1, lon1, lat2, lon2)):
        print('thread_id:', cuda.threadIdx.x, 'bid:', cuda.blockIdx.x,
              'array size:', lat1.size, 'block threads:', cuda.blockDim.x)

        x_1 = pi/180 * x_1
        y_1 = pi/180 * y_1
        x_2 = pi/180 * x_2
        y_2 = pi/180 * y_2
        
        dlon = y_2 - y_1
        dlat = x_2 - x_1
        a = sin(dlat/2)**2 + cos(x_1) * cos(x_2) * sin(dlon/2)**2
        
        c = 2 * asin(sqrt(a)) 
        r = 6371 # Radius of earth in kilometers
        
        out[i] = c * r

In [4]:
outdf = df.apply_chunks(haversine_distance_kernel,
                        incols=['lat1', 'lon1', 'lat2', 'lon2'],
                        outcols=dict(out=np.float64),
                        kwargs=dict(),
                        chunks=16,
                        tpb=8)

In [5]:
print(outdf.head())

                lat1               lon1               lat2               lon2                out
0 10.472985831489979 11.889149236424382 10.173868999704066  9.557544882003544 257.22412709035154
1  9.318574120560555 11.733838880199368 10.237408418960612 10.941703071688115 134.06359643450293
2 10.242439496690478  9.969705766548733  8.158691857367202  8.084420608064908 310.65041668179356
3  8.299264365961669  10.28430047739249 10.376267158128183  9.831747409666285  236.2291524258287
4   10.7531428339492 11.003470626251172  9.416266513336655 12.702623243748901 238.11529748873042


In [ ]:
# Note: print statements in kernels will only appear in terminal output; Jupyter Notebooks won't display them
# Sample print statement output:
tid: 0 bid: 2 array size: 16 block threads: 8
tid: 1 bid: 2 array size: 16 block threads: 8
...
tid: 6 bid: 6 array size: 4 block threads: 8
tid: 7 bid: 6 array size: 4 block threads: 8
...
tid: 6 bid: 1 array size: 16 block threads: 8
tid: 7 bid: 1 array size: 16 block threads: 8
...